In [22]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv("../data/Bakery sales.csv")

# Borrar todas las filas donde la columna 'article' no tenga más de 2 caracteres
df = df[df['article'].str.len() > 2]

# Mostrar las primeras filas del DataFrame resultante
print(df.head())

   Unnamed: 0        date   time  ticket_number               article  \
0           0  2021-01-02  08:38       150040.0              BAGUETTE   
1           1  2021-01-02  08:38       150040.0      PAIN AU CHOCOLAT   
2           4  2021-01-02  09:14       150041.0      PAIN AU CHOCOLAT   
3           5  2021-01-02  09:14       150041.0                  PAIN   
4           8  2021-01-02  09:25       150042.0  TRADITIONAL BAGUETTE   

   Quantity unit_price  
0       1.0     0,90 €  
1       3.0     1,20 €  
2       2.0     1,20 €  
3       1.0     1,15 €  
4       5.0     1,20 €  


In [23]:
# Calcular la demanda valorizada = Quantity * unit_price
"""
   Quantity unit_price  
0       1.0     0,90 €  
1       3.0     1,20 €  
2       2.0     1,20 €  
3       1.0     1,15 €  
4       5.0     1,20 €  

"""
df["DemandaValorizada"] = df["Quantity"] * df["unit_price"].str.replace(" €", "").str.replace(",", ".").astype(float)

# ordenar el DataFrame por la nueva columna, de forma descendente
df = df.sort_values(by="DemandaValorizada", ascending=False)

# Mostrar las primeras filas del DataFrame con la nueva columna
print(df.head())

print(df.info())

        Unnamed: 0        date   time  ticket_number           article  \
50552       110375  2021-06-12  09:58       179931.0       CAFE OU EAU   
74999       163377  2021-07-31  12:32       194199.0        GD NANTAIS   
25378        55521  2021-04-04  12:27       165104.0          ROYAL 6P   
199061      434997  2022-07-23  10:57       268189.0  PLAQUE TARTE 25P   
223128      487139  2022-08-28  08:47       282223.0  FORMULE SANDWICH   

        Quantity unit_price  DemandaValorizada  
50552      200.0     1,00 €              200.0  
74999       11.0    11,00 €              121.0  
25378        4.0    18,00 €               72.0  
199061       2.0    35,00 €               70.0  
223128      10.0     7,00 €               70.0  
<class 'pandas.core.frame.DataFrame'>
Index: 234000 entries, 50552 to 50553
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         234000 non-null  int64  
 1   d

In [24]:
# Agrupar los prudctos por artículo y sumar la demanda valorizada
df_agrupado = df.groupby("article")["DemandaValorizada"].sum().reset_index()
df_agrupado = df_agrupado.sort_values(by="DemandaValorizada", ascending=False)

print(df_agrupado.head())


                  article  DemandaValorizada
139  TRADITIONAL BAGUETTE          144756.05
56       FORMULE SANDWICH           34710.50
31              CROISSANT           33579.50
84       PAIN AU CHOCOLAT           31100.45
6                 BANETTE           24555.15


In [25]:
# Calcular la demanda valorizada porcentual
total = df_agrupado["DemandaValorizada"].sum()
df_agrupado["DemandaValorizadaPorcentual"] = df_agrupado["DemandaValorizada"] / total * 100

print(df_agrupado.head())

                  article  DemandaValorizada  DemandaValorizadaPorcentual
139  TRADITIONAL BAGUETTE          144756.05                    25.912185
56       FORMULE SANDWICH           34710.50                     6.213384
31              CROISSANT           33579.50                     6.010928
84       PAIN AU CHOCOLAT           31100.45                     5.567164
6                 BANETTE           24555.15                     4.395516


In [26]:
# Calcular la demanda valorizada porcentual acumulada respecto a la demanda valorizada
df_agrupado["DemandaValorizadaPorcentualAcumulada"] = df_agrupado["DemandaValorizadaPorcentual"].cumsum()

# Mostrar las primeras filas del DataFrame con la nueva columna
print(df_agrupado.head())

                  article  DemandaValorizada  DemandaValorizadaPorcentual  \
139  TRADITIONAL BAGUETTE          144756.05                    25.912185   
56       FORMULE SANDWICH           34710.50                     6.213384   
31              CROISSANT           33579.50                     6.010928   
84       PAIN AU CHOCOLAT           31100.45                     5.567164   
6                 BANETTE           24555.15                     4.395516   

     DemandaValorizadaPorcentualAcumulada  
139                             25.912185  
56                              32.125569  
31                              38.136497  
84                              43.703660  
6                               48.099177  


In [27]:
# Asignar el tipo A, B, C según la demanda valorizada porcentual acumulada
def asignar_tipo(valor):
    if valor <= 80:
        return 'A'
    elif valor <= 95:
        return 'B'
    else:
        return 'C'

df_agrupado["Tipo"] = df_agrupado["DemandaValorizadaPorcentualAcumulada"].apply(asignar_tipo)

# Mostrar las primeras filas del DataFrame con la nueva columna
print(df_agrupado.head())

# Guardar el DataFrame en un archivo CSV
df_agrupado.to_csv("../data/Bakery sales processed.csv", index=False)

                  article  DemandaValorizada  DemandaValorizadaPorcentual  \
139  TRADITIONAL BAGUETTE          144756.05                    25.912185   
56       FORMULE SANDWICH           34710.50                     6.213384   
31              CROISSANT           33579.50                     6.010928   
84       PAIN AU CHOCOLAT           31100.45                     5.567164   
6                 BANETTE           24555.15                     4.395516   

     DemandaValorizadaPorcentualAcumulada Tipo  
139                             25.912185    A  
56                              32.125569    A  
31                              38.136497    A  
84                              43.703660    A  
6                               48.099177    A  


In [28]:
# Analisis XYZ
# Calcular la columna "month"
df["month"] = df["date"].str.split("-").str[1]

# Para cada articulo calcular la demanda total (no valorizada) por mes
df_agrupado_mes = df.groupby(["article", "month"])["Quantity"].sum().reset_index()

print(df_agrupado_mes.head())

# guardar
df_agrupado_mes.to_csv("../data/Bakery sales processed by month.csv", index=False)



In [29]:
# Rellenar los meses que faltan a cada articulo con 0
df_agrupado_mes = df_agrupado_mes.set_index(["article", "month"]).unstack().fillna(0).stack().reset_index()

print(df_agrupado_mes.head())

# guardar
df_agrupado_mes.to_csv("../data/Bakery sales processed by month filled.csv", index=False)

In [31]:
# Siguiendo con el analisis XYZ, en un nuevo datafreme pone los meses como columnas y los articulos como filas
df_pivot = df_agrupado_mes.pivot(index="article", columns="month", values="Quantity").reset_index()

print(df_pivot.head())

In [34]:
# Calcula el promedio de la demanda por articulo y la desviacion estandar
df_pivot["mean"] = df_pivot.iloc[:, 1:].mean(axis=1)

df_pivot["std"] = df_pivot.iloc[:, 1:].std(axis=1)

print(df_pivot.head())

# guardar
df_pivot.to_csv("../data/Bakery sales std.csv", index=False)

In [36]:
# Asignar XYZ =SI(H3<=10%;"X";SI(H3<=25%;"Y";SI(H3>=25%;"Z")))
def asignar_xyz(valor):
    if valor <= 10:
        return 'X'
    elif valor <= 25:
        return 'Y'
    else:
        return 'Z'
    
df_pivot["XYZ"] = df_pivot["std"].apply(asignar_xyz)

print(df_pivot.head())

# guardar
df_pivot.to_csv("../data/Bakery sales XYZ.csv", index=False)

In [38]:
# contar cuantos articulos hay de cada tipo
df_xyz = df_pivot["XYZ"].value_counts().reset_index()

print(df_xyz)